In [1]:
import pandas as pd

In [2]:
airports = pd.read_csv('airport-codes.csv', sep=',')[['iata_code', 'iso_country', 'coordinates']]
airports = pd.concat([airports.drop(columns='coordinates'),
                      airports.coordinates.str.split(', ', expand=True)], axis=1)
airports = airports[~airports.iata_code.isnull()].rename(columns={0:'lon',1:'lat', 'iata_code':'code', 'iso_country':'country'})

def save_df(airlines, name, coloring):
    df = pd.read_csv('routes.dat', sep='\t', usecols=['airline','A','B']).drop_duplicates()
    df = df[df.airline.isin(airlines)]
    df = df.merge(airports, how='left', left_on='A', right_on='code').merge(airports, how='left', left_on='B', right_on='code', suffixes=('_A','_B'))
    df = df.drop(columns=['code_A','code_B'])
    df['color'] = df.apply(coloring, axis=1)
    df['AB'] = df.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

    df = df[(df.country_A=='US') & (df.country_B=='US')]

    df = df.sample(frac=1)
    df.to_csv(f'{name}.csv', index=False)
    return df

# All

In [3]:
%%time
def coloring(row):
    if row.airline == 'AA': return '#00ffff'
    if row.airline == 'DL': return '#ff0000'
    if row.airline == 'UA': return '#426ae1'

save_df(['AA', 'DL', 'UA'], 'all', coloring)

CPU times: user 116 ms, sys: 7.04 ms, total: 123 ms
Wall time: 123 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
127,AA,CLT,RDU,US,-80.94309997558594,35.2140007019043,US,-78.7874984741211,35.877601623535156,#00ffff,CLT_RDU
1562,DL,LGA,TPA,US,-73.872597,40.777199,US,-82.533203125,27.975500106811523,#ff0000,LGA_TPA
554,AA,LGA,PIT,US,-73.872597,40.777199,US,-80.23290253,40.49150085,#00ffff,LGA_PIT
2420,UA,ORD,AVL,US,-87.9048,41.9786,US,-82.541801,35.436199,#426ae1,ORD_AVL
1032,DL,ATL,DTW,US,-84.428101,33.6367,US,-83.35340118408203,42.212398529052734,#ff0000,ATL_DTW
...,...,...,...,...,...,...,...,...,...,...,...
509,AA,LAX,OMA,US,-118.407997,33.942501,US,-95.894096,41.3032,#00ffff,LAX_OMA
1792,DL,SLC,RNO,US,-111.97799682617188,40.78839874267578,US,-119.76799774169922,39.49909973144531,#ff0000,SLC_RNO
955,AA,PHX,SBA,US,-112.01200103759766,33.43429946899414,US,-119.8399963,34.42620087,#00ffff,PHX_SBA
1931,UA,DEN,PDX,US,-104.672996521,39.861698150635,US,-122.5979996,45.58869934,#426ae1,DEN_PDX


# American

In [4]:
%%time
def coloring(row):
    hubs = ['CLT','ORD','DFW','LAX','MIA','LGA','PHL', 'PHX', 'DCA']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['AA'], 'american', coloring)

CPU times: user 79.6 ms, sys: 6.12 ms, total: 85.7 ms
Wall time: 83.9 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
723,AA,ORD,GRR,US,-87.9048,41.9786,US,-85.52279663,42.88079834,#ffff00,ORD_GRR
140,AA,CLT,SFO,US,-80.94309997558594,35.2140007019043,US,-122.375,37.61899948120117,#ff0000,CLT_SFO
785,AA,ORD,TPA,US,-87.9048,41.9786,US,-82.533203125,27.975500106811523,#ffff00,ORD_TPA
712,AA,ORD,DSM,US,-87.9048,41.9786,US,-93.66310119628906,41.534000396728516,#ffff00,ORD_DSM
55,AA,CLT,FAY,US,-80.94309997558594,35.2140007019043,US,-78.88030242919922,34.9911994934082,#ff0000,CLT_FAY
...,...,...,...,...,...,...,...,...,...,...,...
745,AA,ORD,MHT,US,-87.9048,41.9786,US,-71.435699,42.932598,#ffff00,ORD_MHT
103,AA,CLT,MSP,US,-80.94309997558594,35.2140007019043,US,-93.221802,44.882,#ff0000,CLT_MSP
495,AA,LAX,IAH,US,-118.407997,33.942501,US,-95.34140014648438,29.984399795532227,#00ffff,LAX_IAH
353,AA,DFW,MCI,US,-97.038002,32.896801,US,-94.713898,39.2976,#ffa500,DFW_MCI


# Delta

In [5]:

def coloring(row):
    hubs = ['ATL','BOS','DTW','JFK','LAX','LGA','MSP','SEA','SLC']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#007f00', '#ffddad', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['DL'], 'delta', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
675,DL,MSP,RAP,US,-93.221802,44.882,US,-103.05699920654297,44.0452995300293,#fd68b3,MSP_RAP
268,DL,CVG,DCA,US,-84.667801,39.048801,US,-77.037697,38.8521,gray,CVG_DCA
724,DL,SEA,ANC,US,-122.308998,47.449001,US,-149.99600219726562,61.174400329589844,#426ae1,SEA_ANC
634,DL,MSP,GRB,US,-93.221802,44.882,US,-88.12960052490234,44.48509979248047,#fd68b3,MSP_GRB
784,DL,SLC,FLL,US,-111.97799682617188,40.78839874267578,US,-80.152702,26.072599,#6cff7d,SLC_FLL
...,...,...,...,...,...,...,...,...,...,...,...
592,DL,LGA,TYS,US,-73.872597,40.777199,US,-83.9940033,35.81100082,#ffddad,LGA_TYS
182,DL,ATL,SFO,US,-84.428101,33.6367,US,-122.375,37.61899948120117,#ff0000,ATL_SFO
318,DL,DTW,CIU,US,-83.35340118408203,42.212398529052734,US,-84.47239685058594,46.25080108642578,#ffa500,DTW_CIU
181,DL,ATL,SEA,US,-84.428101,33.6367,US,-122.308998,47.449001,white,ATL_SEA


# United

In [6]:

def coloring(row):
    hubs = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['UA'], 'united', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
101,UA,DEN,MIA,US,-104.672996521,39.861698150635,US,-80.29060363769531,25.79319953918457,#ffff00,DEN_MIA
147,UA,DEN,SGF,US,-104.672996521,39.861698150635,US,-93.38860321,37.24570084,#ffff00,DEN_SGF
695,UA,ORD,MTJ,US,-87.9048,41.9786,US,-107.893997192,38.509799957300004,#ff0000,ORD_MTJ
626,UA,ORD,CGI,US,-87.9048,41.9786,US,-89.57080078125,37.22529983520508,#ff0000,ORD_CGI
743,UA,ORD,TPA,US,-87.9048,41.9786,US,-82.533203125,27.975500106811523,#ff0000,ORD_TPA
...,...,...,...,...,...,...,...,...,...,...,...
173,UA,DVL,JMS,US,-98.90879822,48.11420059,US,-98.67819977,46.92969894,gray,DVL_JMS
347,UA,IAD,LAX,US,-77.45580292,38.94449997,US,-118.407997,33.942501,white,IAD_LAX
80,UA,DEN,IND,US,-104.672996521,39.861698150635,US,-86.294403,39.7173,#ffff00,DEN_IND
762,UA,SFO,BFL,US,-122.375,37.61899948120117,US,-119.0569992,35.43360138,#426ae1,SFO_BFL
